# FFSC Rocket Engine and Regenerative Nozzle Cooling Model

This notebook documents:

1. The **governing equations** for a 1D regenerative nozzle cooling model.
2. A simplified **full-flow staged combustion (FFSC)** cycle model.
3. A Python implementation in the `ffsc_nozzle` package.
4. Example **optimizations and parameter sweeps** using that package.

We assume a methane–oxygen engine (CH₄/LOX), but the structure can be generalized.

## Governing Equations

### Nozzle gas side (1D isentropic + Bartz)

We approximate the core flow as isentropic:

- Area–Mach relation
- Static temperature:
  $$
  T(x) = \frac{T_0}{1 + \frac{\gamma - 1}{2} M(x)^2}
  $$
- Static pressure:
  $$
  p(x) = p_0 \left( \frac{T(x)}{T_0} \right)^{\gamma / (\gamma - 1)}
  $$

We use a Bartz-type correlation for the gas-side convective heat transfer coefficient:
$$
h_g(x) \approx C \,
\frac{\mu_g^{0.2} c_{p,g}^{0.6}}{\mathrm{Pr}_g^{0.6}}\,
p_0^{0.8} r_t^{0.1}\left( \frac{A_t}{A(x)} \right)^{0.9}
\left( \frac{T_0}{T(x)} \right)^{0.55}
$$

### Coolant side (Dittus–Boelter / Gnielinski + Sieder–Tate)

For each axial station, we treat the coolant channel as a turbulent internal flow:

- Hydraulic diameter:
  $$
  D_h = \frac{4 A_c}{P_{\text{wet}}}
  $$
- Reynolds and Prandtl:
  $$
  \mathrm{Re} = \frac{G D_h}{\mu}, \quad \mathrm{Pr} = \frac{c_p \mu}{k}
  $$

We use the Gnielinski correlation:
$$
\mathrm{Nu} =
\frac{(f/8)(\mathrm{Re}-1000)\mathrm{Pr}}{
1 + 12.7 \sqrt{f/8} (\mathrm{Pr}^{2/3} - 1)
}
$$

with a Sieder–Tate viscosity correction:
$$
\mathrm{Nu} \to \mathrm{Nu} \left( \frac{\mu_b}{\mu_w} \right)^{0.14}
$$

Then:
$$
h_c = \frac{\mathrm{Nu}\,k}{D_h}.
$$

### Wall conduction and conjugate heat transfer

We treat the wall as a 1D radial conduction layer:

- Inner gas film, wall, and coolant film form series resistances:
  $$
  R_{\text{tot}} = \frac{1}{h_g} + \frac{t_{\text{wall}}}{k_{\text{wall}}} + \frac{1}{h_c}
  $$
- Heat flux:
  $$
  q''(x) = \frac{T_g(x) - T_{\text{cool}}(x)}{R_{\text{tot}}}
  $$
- Inner wall temperature:
  $$
  T_{\text{wall,in}}(x) = T_g(x) - \frac{q''(x)}{h_g}
  $$

### Coolant energy equation

We integrate the coolant energy along the axial coordinate:

$$
\dot{m}_{\text{cool}} c_{p,c} \frac{d T_{\text{cool}}}{dx}
  = q''(x) P_{\text{inner}}(x)
$$

which we discretize:
$$
T_{\text{cool}, i+1} = T_{\text{cool}, i}
+ \frac{q''_i P_{\text{inner}, i} \Delta x}{\dot{m}_{\text{cool}} c_{p,c,i}}.
$$

### FFSC cycle: Pumps and turbines

- Pump power:
  $$
  P_{\text{pump}} = \frac{\dot{m} \Delta p}{\rho \eta_{\text{pump}}}
  $$
- Turbine power:
  $$
  P_{\text{turb}} = \dot{m} c_p (T_{\text{in}} - T_{\text{out}})\eta_{\text{turb}}
  $$

We require:
$$
P_{\text{turb, fuel}} \gtrsim \frac{P_{\text{pump, fuel}}}{\eta_{\text{mech}}},\quad
P_{\text{turb, ox}}   \gtrsim \frac{P_{\text{pump, ox}}}{\eta_{\text{mech}}}.
$$

### Main chamber thermodynamics

If Cantera is available, we compute an equilibrium CH₄/O₂ state at $p_0$ and O/F:

- $T_0$, $\gamma$, $R_g$ from equilibrium composition.
- This feeds both the nozzle model (for $h_g$) and the performance estimate.

Otherwise, we fall back to a parameterized ideal-gas guess.

In [ ]:
# Import the required python packages
import numpy as np
import matplotlib.pyplot as plt

from ffsc_nozzle import thermo, regen, cycle, sweep

# Check optional tools
print("Cantera available:", thermo.HAVE_CANTERA)
try:
    from CoolProp.CoolProp import PropsSI
    print("CoolProp available: True")
    HAVE_COOLPROP = True
except ImportError:
    print("CoolProp available: False")
    HAVE_COOLPROP = False

In [ ]:
p0 = 20e6    # Pa
OF = 3.2

if thermo.HAVE_CANTERA:
    T0, gamma, R_g, gas_ch = thermo.cantera_chamber_state(
        OF=OF,
        p0=p0,
        T_fuel=110.0,
        T_ox=90.0,
        fuel_species="CH4",
        ox_species="O2",
        mech="gri30.yaml",
    )
    print(f"Chamber from Cantera: T0={T0:.1f} K, gamma={gamma:.3f}, R_g={R_g:.1f} J/kg/K")
else:
    T0, gamma, R_g = thermo.ideal_gas_chamber_state(OF=OF, p0=p0)
    print(f"Chamber (fallback): T0={T0:.1f} K, gamma={gamma:.3f}, R_g={R_g:.1f} J/kg/K")

In [ ]:
# %% 
r_t = 0.15
eps = 15.0
L_noz = 1.8

x, r, A, At = regen.simple_conical_nozzle(r_t, eps, L_noz, N=300)

# Use Cantera chamber if available
if thermo.HAVE_CANTERA:
    T0, gamma, R_g, _ = thermo.cantera_chamber_state(OF=OF, p0=p0)
else:
    T0, gamma, R_g = thermo.ideal_gas_chamber_state(OF=OF, p0=p0)

# crude mass flows for demonstration
F_vac = 1.0e6
from ffsc_nozzle.sweep import ideal_vacuum_isp_from_eps
Isp_vac_ideal, _, T_e, M_e = ideal_vacuum_isp_from_eps(p0, T0, gamma, R_g, eps)
Isp_eff = 0.95*Isp_vac_ideal
g0 = 9.80665
m_dot_total = F_vac/(Isp_eff*g0)
m_dot_fuel = m_dot_total/(1+OF)
m_dot_cool = 0.2*m_dot_fuel

res_noz = regen.regen_nozzle_1D(
    x=x, r=r, A=A, At=At,
    p0=p0, T0=T0, gamma=gamma, R_g=R_g,
    coolant="LCH4",
    coolant_props=regen.make_coolprop_liquid("Methane") if HAVE_COOLPROP else None,
    m_dot_cool=m_dot_cool,
    n_channels=120,
    w_channel=0.0015,
    h_channel=0.0020,
    wall_thickness=0.0020,
    roughness=5e-6,
    T_cool_in=110.0,
    p_cool_in=18e6,
)

print("Max wall temperature:", res_noz["T_wall_inner"].max(), "K")

plt.figure(figsize=(7,4))
plt.plot(res_noz["x"], res_noz["T_wall_inner"], label="T_wall_inner")
plt.plot(res_noz["x"], res_noz["T_cool"], label="T_cool")
plt.xlabel("x [m]")
plt.ylabel("Temperature [K]")
plt.legend()
plt.title("Nozzle cooling (single pass)")
plt.tight_layout()
plt.show()

In [ ]:
# %% 
summary = cycle.ffsc_full_flow_cycle(
    F_vac=F_vac,
    p0=p0,
    OF=OF,
    r_t=r_t,
    eps=eps,
    L_noz=L_noz,
    coolant="LCH4",
    use_coolprop=True,
    use_cantera_chamber=True,
)

for k, v in summary.items():
    if isinstance(v, (int, float)):
        print(f"{k:20s} : {v}")

In [ ]:
# %% 
p0_array = np.linspace(10e6, 30e6, 5)   # 10–30 MPa
OF_array = np.linspace(2.5, 4.0, 7)     # O/F 2.5–4.0

results = sweep.sweep_ffsc_feasibility(
    F_vac=F_vac,
    r_t=r_t,
    eps=eps,
    L_noz=L_noz,
    p0_array=p0_array,
    OF_array=OF_array,
)

P0 = results["p0_grid"]/1e6   # MPa
OFG = results["OF_grid"]
feas = results["feasible"]

plt.figure(figsize=(6,5))
cs = plt.contourf(
    OFG, P0,
    feas.astype(float),
    levels=[-0.1,0.5,1.1],
)
plt.xlabel("O/F")
plt.ylabel("Chamber pressure p0 [MPa]")
plt.title("FFSC feasibility map (1 = feasible)")
plt.colorbar(label="Feasible")
plt.tight_layout()
plt.show()